In [1]:
import pandas as pd

!pip install psycopg2
import psycopg2

In [2]:
# Nome do arquivo Excel
excel_file = 'C:/Users/thiago.silva/TCC/RAIS/RAIS_estabelecimento_layout2018e2019.xlsx'

# Leitura do arquivo Excel
excel_data = pd.ExcelFile(excel_file)

# Extrai os nomes das abas em uma lista
aba_names = excel_data.sheet_names

# Imprime os nomes das abas
print(aba_names)

['ESTB - layout', 'BAIRRO_SP', 'BAIRRO FORT', 'BAIRRO_RJ', 'DISTRITO_SP', 'REG ADM DF', 'municipio', 'subclasse 2.0', 'classe 1.0 ou 95']


In [15]:
# Configurações do banco de dados PostgreSQL
db_config = {
    "host"      : "dataiesb.iesbtech.com.br",
    "database"  : "IESB_Empresas",
    "user"      : "1922120026_TCC_Thiago",
    "password"  : "********"}

# Dicionário com as correspondências entre abas e sufixos
sufixos1 = {'BAIRRO_SP': 'bairro_sp',
           'BAIRRO FORT': 'bairro_fortaleza',
           'BAIRRO_RJ': 'bairro_rio',
           'DISTRITO_SP': 'distrito_sp',
           'REG ADM DF': 'ra_df'}
sufixos2 = {'municipio': 'municipio',
           'subclasse 2.0': 'subclasse_cnae2',
           'classe 1.0 ou 95': 'classe1_95'}

# Função para criar uma tabela no banco de dados
def create_table1(cursor, table_name):
    create_table_query = f"CREATE TABLE {table_name} (categorias TEXT, descricao TEXT, valor_fonte TEXT);"
    cursor.execute(create_table_query)
    
# Função para criar uma tabela no banco de dados
def create_table2(cursor, table_name):
    create_table_query = f"CREATE TABLE {table_name} (codigo TEXT, descricao TEXT);"
    cursor.execute(create_table_query)

    # Conexão com o banco de dados
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

# Leitura do arquivo Excel
excel_data = pd.read_excel(excel_file, sheet_name=None)

# Itera sobre as abas do arquivo Excel
for sheet_name, data_frame in excel_data.items():
    # Verifica se há um sufixo definido para a aba
    if sheet_name in sufixos1:
        print(f'Lendo aba {sheet_name}...')
        sufixo = sufixos1[sheet_name]
        table_name = f'tb_rais_desc_{sufixo}'

        # Criação da tabela no banco de dados
        print(f'Criando tabela {table_name} no postgres...')
        create_table1(cursor, table_name)
        conn.commit()

        # Itera sobre as colunas
        for _, row in data_frame.iterrows():
            categorias = row['Categorias']
            descricao = row['Descrição']
            valor_fonte = row['Valor na Fonte']

            # Inserção dos dados na tabela
            insert_query = f"INSERT INTO {table_name} (categorias, descricao, valor_fonte) VALUES (%s, %s, %s);"
            cursor.execute(insert_query, (categorias, descricao, valor_fonte))
            conn.commit()
        print('Dados inseridos no banco com sucesso!')

for sheet_name, data_frame in excel_data.items():
    # Verifica se há um sufixo definido para a aba
    if sheet_name in sufixos2:
        print(f'Lendo aba {sheet_name}...')
        sufixo = sufixos2[sheet_name]
        table_name = f'tb_rais_desc_{sufixo}'

        # Criação da tabela no banco de dados
        print(f'Criando tabela {table_name} no postgres...')
        create_table2(cursor, table_name)
        conn.commit()

        # Itera sobre as colunas
        for _, row in data_frame.iterrows():
            codigo = row['Código']
            descricao = row['Descrição']

            # Inserção dos dados na tabela
            insert_query = f"INSERT INTO {table_name} (codigo, descricao) VALUES (%s, %s);"
            cursor.execute(insert_query, (codigo, descricao))
            conn.commit()
        print('Dados inseridos no banco com sucesso!')

# Fechamento da conexão com o banco de dados
cursor.close()
conn.close()

Lendo aba BAIRRO_SP...
Criando tabela tb_rais_desc_bairro_sp no postgres...
Dados inseridos no banco com sucesso!
Lendo aba BAIRRO FORT...
Criando tabela tb_rais_desc_bairro_fortaleza no postgres...
Dados inseridos no banco com sucesso!
Lendo aba BAIRRO_RJ...
Criando tabela tb_rais_desc_bairro_rio no postgres...
Dados inseridos no banco com sucesso!
Lendo aba DISTRITO_SP...
Criando tabela tb_rais_desc_distrito_sp no postgres...
Dados inseridos no banco com sucesso!
Lendo aba REG ADM DF...
Criando tabela tb_rais_desc_ra_df no postgres...
Dados inseridos no banco com sucesso!
Lendo aba municipio...
Criando tabela tb_rais_desc_municipio no postgres...
Dados inseridos no banco com sucesso!
Lendo aba subclasse 2.0...
Criando tabela tb_rais_desc_subclasse_cnae2 no postgres...
Dados inseridos no banco com sucesso!
Lendo aba classe 1.0 ou 95...
Criando tabela tb_rais_desc_classe1_95 no postgres...
Dados inseridos no banco com sucesso!
